<a href="https://colab.research.google.com/github/opopiol/ML_project/blob/model/raport.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **1. EDA**

After checking
*   there is no problem with missing values
*   the data is almost normally distributed

We also change -1 to 0, because it was easier to visualize data later.

There have been loaded two scalers `StandardScaler()` and `MinMaxScaler()`, but we will be using only `StandardScaler()` later. Data scaling is a recommended pre-processing step when working with many machine learning algorithms.

It is used in scatterplot, which started a tries of proper clustering. We are using `PCA()` to reduce the dimension of features. But none of the use of the PCA itself has worked out.

Later the first pipeline was created. It included apart from `PCA()` also `TSNE()`. It turns out that best for our project will be n_components=0.99 for PCA and n_components=2 for TSNE, because we can clearly see how the data grouped in unsupervised way. 



For splitting data we use: `X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify = y, random_state=1)`. Splitted dataset: 70% is used to train the model and 30% to evaluate it. `stratify=y` will keep proportion of values in the sample produced, the same as the proportion of values provided to parameter stratify.

### **2. Metrics**

In [ ]:
y.value_counts()

y 
 1    3375
-1     375
dtype: int64

The distribution of the labels is very uneven. They appear in proportions of about 1:10.

That’s why we decided to choose f1-score because it fits the best to our dataset and deals best with disproportion. This metric is the harmonic mean of the both precision and recall.

And because we have to predict the class of given data we use classifiers. 

### **3. Baseline**

For the base model we choose `DummyClassifier()`, beacuase it is a simple baseline for the other classifiers. 

Strategies we used are `['stratified', 'most_frequent', 'constant', 'uniform']` and we get the following results:

*   tratified strategy f1 score is 0.8951813214108296
*   most_frequent strategy f1 score is 0.9476145930776426
*   prior strategy f1 score is 0.9476145930776426
*   uniform strategy f1 score is 0.6650093225605965


The highest results were achived by `most_frequent` strategy and `prior` strategy with a result above 94%. But both of them were only using one, majority class.

For another baseline, we decided to check also `DecisionTreeClassifier()` and its f1 score was above 96%.

### **4. Model**

In the first approach to choosing the best class factor we decided to check the following three classifiers `LogisticRegression(random_state=0, max_iter = 1000)`, `KNeighborsClassifier()` and `SVC()`.

Their f1 scores were:

*   `LogisticRegression` 0.891662506240639
*   `KNeighborsClassifier` 0.9803921568627452
*   `SVC` 0.9767211490837047

Because `LogisticRegression` achieved a result lower than the base, it was not taken into account further.

`KNeighborsClassifier`, with its default number of 5 neighbors, performed the best out of three given classifiers. Also model is usually robust to noisy data.

`SVC` also performed better f1 score than our base model. 


Later we were looking for more advanced calssifiers. Some of the most popular are AdaBoost, Gradient Boosting, and XGBoost. Due to the long processing time, we decided to try only one- `AdaBoostClassifier()` to see what results it would achieve. In its basic model, without the given parameters, it achieved a f1 score 0.9656188605108056, so above baseline score.

> In its basic model, without the given parameters, it achieved a f1 score 0.9656188605108056, so above baseline score.

Then a pipeline was created to see which parameters of `AdaBoostClassifier()` performed best. In the pipeline were used: `'n_estimators': [50, 100, 150], 'learning_rate': [0.1, 0.01, 0.05]`. It was done in order not to overload the fan pipeline so much. 

> It turns out that the best parameters was `algorithm='SAMME.R', base_estimator=None, learning_rate=0.05, n_estimators=50, random_state=None`. The model achieved a score of 0.9793510324483775.

These params were used in final pipeline with others classifiers.

The final pipeline had parameters and models choosen earlier.

`PCA()` got n_components = 0.99, because with this params it performed the best in EDA's clustering.
As scaler we got `StandardScaler()` same as in EDA and does not requires any parameters to be optimised by GridSearchCV. 

In pipeline three calssifier were checked: 

*   `SVC()` with `kernel: ['linear', 'poly'], class_weight: ['balanced'], and C: np.logspace(1,4,5)`
*   `KNeighborsClassifier()` with `n_neighbors': [2, 4, 6, 8, 10], algorithm': ['auto']`
*   `AdaBoostClassifier()` with the parameters that came out in the previous pipeline `n_estimators': [50], learning_rate': [0.1]`

We used GridSearchCV to select the best model. To avoid overfitting, instead of the data that was used for training, we used `test_data`.


> The result of `best_model.best_estimator_` was `'classifier', AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.1, n_estimators=50, random_state=None)`. 
Again `AdaBoostClassifier` was choosed as the best model and this is our final choice. 